# **Setup**

GDAL
Python
Tesnorflow-GPU
Get list of dpendcies (Clone VE)
explain helpers
explain VE activation
explain runnung python commends

# **Tiles**

The tiles are created using GDAL.
The following GDAL codes consist of 2 parts:
1. **Combining the RGB raster and the labels raster into one combined raster**. This is the most efficient way, however, you can divide the RGB raster and the labels raster into tiles separately, and then combine them later on in the process (Using a script I will attach later on), although as I said, creating the raster in this step is the preferred way to do it.

2. **Generate tiles from the combined images.**







## **Combining The Rasters**

Run the following commands in terminal (We used Annconda):
1. Change the directoy to your working environment:

  `cd /home/michael/downloads/generalization/`
2. Process all 3 bands of the RGB image (If necessary, Change `image.tif` to the name of the RGB image you want to process):

  a. `gdal_translate -b 1 -mask 4 -co COMPRESS=LZW -co BIGTIFF=YES --config GDAL_TIFF_INTERNAL_MASK YES image.tif image_band1.tif`

  b.  ` gdal_translate -b 2 -mask 4 -co COMPRESS=LZW -co BIGTIFF=YES --config GDAL_TIFF_INTERNAL_MASK YES image.tif image_band2.tif`

  c.  ` gdal_translate -b 3 -mask 4 -co COMPRESS=LZW -co BIGTIFF=YES --config GDAL_TIFF_INTERNAL_MASK YES image.tif image_band3.tif`




3. Generate a virtual raster of the combined RGB bands and lables band (If necessary, Change `labels.tif` to the name of the labels image you want to process)
 `gdalbuildvrt -separate combined.vrt image_band1.tif image_band2.tif image_band3.tif labels.tif`

4.  Process the virtual raster to an actual tif file:
`gdal_translate -co COMPRESS=LZW -co BIGTIFF=YES --config GDAL_TIFF_INTERNAL_MASK YES combined.vrt combined.tif`


## **Generate Tiles**

Adjust the parameters and working envorimmnet (if you haven't done it in the last part) and **only then** run the following commend in terminal (again, we used Annconda):

`python C:\ProgramData\Miniconda\pkgs\gdal-3.4.1-py39h9b7a543_0\Scripts\gdal_retile.py -co COMPRESS=LZW -co "TILED=YES" -ps 256 256 -overlap 5 -targetDir tiles_no_overlap -tileIndex index combined.tif`


**Parameters**:


*   `-ps` = Number of pixels in each tile
*   `-overlap` = Pixel overlap of the tiles. For example- for an overlap of 50% of a 256x256 tile with the previous 256x256 tile, the value of this parameter should be 128 (256/2).
*   `-targetDir` = The directory the tiles will be saved at. Make sure this directory exists **before** you run the code.
*   `-tileIndex index` = The raster you want to divide.

For more information about the parmaters of this script go to https://gdal.org/programs/gdal_retile.html.








# **Training**

## **Helpers**

Helpers class should be in the same working directory as the python code.
There is no need to adjust or run the helpers code.

In [ ]:
import numpy as np
import rasterio

## Read image into tuple of RGB [x, y, 1-3] and target [x, y, 4]
def read_image(path):
    src = rasterio.open(path)
    r = src.read()
    r[3][r[3] == 255] = 0  ## Replace 255 with 0 in 4th band
    r = np.moveaxis(r, 0, 2)
    image = r[:, :, :3]
    target = r[:, :, [3]]
    return (image, target)

## Read images into two tuples with arrays [i, x, y, 1-3] and [i, x, y, 4]
def read_images(paths):
    images = [read_image(i) for i in paths]
    images, targets = zip(*images)
    images = np.stack(images)
    targets = np.stack(targets)
    return (images, targets)

## Predict raster (file_in=RGB, file_out=pred:0,1,2)
def predict_raster(model, filename_in, filename_out, size):
    src = rasterio.open(filename_in)
    r = read_image(filename_in)[0]
    if r.shape != size:
        return 0
    if (r.flatten() == 0).all():
        return 0
    mask = model.predict(np.expand_dims(r, 0), verbose=0)[0]
    pred = np.argmax(mask, axis=-1)
    meta = src.meta
    meta.update(count=1)
    dst = rasterio.open(filename_out, 'w', **meta)
    dst.write(pred, 1)
    dst.close()
    src.close()


## **Filter (Optional)**

The folllwoing code filters the tiles dataset and keeps only images that contain at least one pixes of a solar panel.


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.plot import show
import glob
import os
import shutil

# Paths
path_in = 'D:/U-Net/training/combined_images/'
path_out = 'D:/U-Net/Segnet/model_input/'

# Minimum number of class=2 (solar panel) pixels
cutoff = 12

# Files Load
files = glob.glob(path_in + '*.tif')

# Filter
for i in files:
  src = rasterio.open(i)
  r = src.read()
  if (r[3] > 0).sum() > cutoff:
    shutil.copy(i, path_out)

## **Generate Train, Test & Validation Datasets**


No need to edit, except relevant paths.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import glob
import random

# Input Path- enter the path of the directory that contains the tiles.
path = 'D:/U-Net/Segnet/model_input/'

# Loads & Sorts Files
files = glob.glob(path + '*.tif')
files.sort()

# Shuffles the order of the tiles
random.Random(7).shuffle(files)

## DS Split to Train, Test & Validation
# Cutoffs
cutoff1 = int(len(files)*0.8) # 80% Training
cutoff2 = cutoff1 + int(len(files)*0.1) # 10% Test & 10% Validation.
# Slicing
files_training = files[:cutoff1]
files_validation = files[cutoff1:cutoff2]
files_test = files[cutoff2:]


## Save- If you are retraining change the name of the txt files or the output path
# Path to save the results
Output_path = "D:/U-Net/Segnet/results/result_files_"

# Train
file = open(Output_path + "training.txt", 'w')
file.writelines('\n'.join(files_training))
file.close()
# Test
file = open(Output_path + "validation.txt", 'w')
file.writelines('\n'.join(files_validation))
file.close()
# Validation
file = open(Output_path + "test.txt", 'w')
file.writelines('\n'.join(files_test))
file.close()


## **Model Training**

### **Segnet architecture**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import glob
import random
import helpers

plt.rcParams['figure.figsize'] = (4, 4)

# Read training files using path (path can be edited)

files = open('D:/U-Net/Segnet/results/result_21_files_training.txt').read().splitlines()

training_images, training_targets = helpers.read_images(files)
print("training images shape:", training_images.shape)
print("raining targets shape:", training_targets.shape)

# Read validation using path (path can be edited)
files = open('D:/U-Net/Segnet/results/result_21_files_validation.txt').read().splitlines()

validation_images, validation_targets = helpers.read_images(files)

# Segnet arichetecture  - optimized for 256X256 tiles.

img_size = training_images[0,:,:,0].shape
def get_model(img_size, num_classes):
    # Encoder
    inputs = keras.Input(shape=img_size + (3,))
    x = layers.Rescaling(1./255)(inputs)
    x = layers.Conv2D(64, (3, 3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)

    x = layers.Conv2D(128, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)

    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.UpSampling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.UpSampling2D((2, 2))(x)

    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.UpSampling2D((2, 2))(x)

    outputs = layers.Conv2D(num_classes, (3, 3), padding='same', activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    return model

model = get_model(img_size=img_size, num_classes=3)
model.summary() #print model summary

# Compile model using rmsprop and loss sparse categorical crossentropy, can be changed according to specific needs
# Also epochs number, callback and batch size can be edited according to the relevant mission
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
callbacks = [keras.callbacks.ModelCheckpoint('solar_panels.keras', save_best_only=True)]
history = model.fit(
    training_images, training_targets,
    epochs=20,
    callbacks=callbacks,
    batch_size=64,
    validation_data=(validation_images, validation_targets)
)

# Save Model to specific path so we can load it after
model.save('D:/U-Net/Segnet/models/model_segnet')

# Plot Model Fitting History to check the performence
epochs = range(1, len(history.history['loss']) + 1)
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig('D:/U-Net/Segnet/history_20Epochs.png')

### **UNET**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import rasterio
import glob
import random
import helpers

## Settings for running Tensorflow with GPU

print(tf.test.is_gpu_available())
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.set_visible_devices(physical_devices[0], 'GPU')

# Get the list of physical devices
physical_devices = tf.config.list_physical_devices()

# Print the list of physical devices
print(physical_devices)

# Check if GPU is in the list of physical devices
if len(physical_devices) > 0:
    for device in physical_devices:
        if device.device_type == 'GPU':
            print('GPU is set as the default device.')
            break
else:
    print('No GPU is available.')

### U-Net

# Defining inputs:

IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3


## Build the U-Net model architecture to fit our input size 256X256


# Load inputs and normalize
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

# Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
p5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c5)

c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p5)
c6 = tf.keras.layers.Dropout(0.3)(c6)
c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c6)

# Expansive path
u7 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c5])
c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c4])
c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.2)(c8)
c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c3])
c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.2)(c9)
c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

u10 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c9)
u10 = tf.keras.layers.concatenate([u10, c2])
c10 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u10)
c10 = tf.keras.layers.Dropout(0.1)(c10)
c10 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c10)

u11 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c10)
u11 = tf.keras.layers.concatenate([u11, c1], axis=3)
c11 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u11)
c11 = tf.keras.layers.Dropout(0.1)(c11)
c11 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c11)

outputs = tf.keras.layers.Conv2D(3, (1, 1), activation='sigmoid')(c11)

# Setting model inputs & outputs

model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

# Printing model summary
model.summary()




# Read training data from path
files = open('D:/U-Net/Segnet/results/result_21_files_training.txt').read().splitlines()

# Open data using helpers
training_images, training_targets = helpers.read_images(files)

# Print shapes
print("training images shape:", training_images.shape)
print("training targets shape:", training_targets.shape)

# Read validation data from path
files = open('D:/U-Net/Segnet/results/result_21_files_validation.txt').read().splitlines()

# Open data using helpers
validation_images, validation_targets = helpers.read_images(files)

# Compile model using rmsprop and loss sparse categorical crossentropy, can be changed according to specific needs
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

# Setting callbacks
callbacks = [keras.callbacks.ModelCheckpoint('solar_panels.keras', save_best_only=True)]

# Train the model
# Also epochs number, callback and batch size can be edited according to the relevant mission

history = model.fit(
    training_images, training_targets,
    epochs=50,
    callbacks=callbacks,
    batch_size=64,
    validation_data=(validation_images, validation_targets)
)

# Save Model to specific path so we can load it after
model.save('D:/U-Net/U-Net_GPU')

# Plot Model Fitting History to check the performence is an option as in Segnet, we decided that it's not relevant in that case.

## **Model Evaluation**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import glob
import helpers
import sklearn.metrics

# Load Model from the path that we saved into
model = keras.models.load_model('D:/U-Net/Segnet/models/model_segnet_v01')

# Read the test files, path can be edited
files = open('D:/U-Net/Segnet/results/result_21_files_test.txt').read().splitlines()

test_images, test_targets = helpers.read_images(files)

# Preidct Test
mask = model.predict(test_images)
pred = np.argmax(mask, axis=-1)
np.unique(pred.flatten())
obs = test_targets[:, :, :, 0]
y_obs = obs.flatten()
y_pred = pred.flatten()

# Confution Matrix
conf_matrix = sklearn.metrics.confusion_matrix(y_obs, y_pred, labels=[0, 1, 2])

# Define the headers and indices
headers = ["Predicted 0", "Predicted 1", "Predicted 2"]
indices = ["Actual 0", "Actual 1", "Actual 2"]

# Print the confusion matrix with headers and indices
print("\t\t" + "\t".join(headers))
for i in range(len(conf_matrix)):
    row_str = "\t".join([str(x) for x in conf_matrix[i]])
    print(indices[i] + "\t" + row_str)

# Calculate true positives (TP), false positives (FP), and false negatives (FN) for each class
TP = np.diag(conf_matrix)
FP = np.sum(conf_matrix, axis=0) - TP
FN = np.sum(conf_matrix, axis=1) - TP

# Calculate precision and recall for each class
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print precision and recall for each class
for i in range(len(precision)):
    print("Class", i)
    print("Precision:", precision[i])
    print("Recall:", recall[i])
    print("F1 score:", f1_score[i])
    print()

# **Generalization**

## **Dataset Creation**

Same as in training but only test set.
Path can be changed according to your paths.



In [ ]:
import numpy as np
import rasterio
import glob
import random

path = 'D:/U-Net/generalization/tiles_no_overlap/'

# Load Files
files = glob.glob(path + '*.tif')
filtered_files = []

for f in files:
    with rasterio.open(f) as src:
        height, width = src.shape
        if height == 256 and width == 256:
            filtered_files.append(f)

print("Number Of Files:", len(filtered_files))

# Write
file = open('D:/U-Net/Segnet/Gen/results/Gen_tiles_no_overlap_dim.txt', 'w')
file.writelines('\n'.join(filtered_files))
file.close()


## **Model Evaluation**

same but runs in batches due to size

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import rasterio
import glob
import helpers
import sklearn.metrics

# Load Model
#model = keras.models.load_model('D:/U-Net/Segnet/models/model_segnet_v01')
model = keras.models.load_model('D:/U-Net/Segnet/models/model_segnet_v06_half_20Epochs')

# Read file paths
files = open('D:/U-Net/Segnet/Gen/results/Gen_tiles_no_overlap_dim.txt').read().splitlines()

# Initialize the cumulative confusion matrix
conf_matrix_cumulative = np.zeros((3, 3))

# Process images in batches, 500 were chosen due to memory considerations.
batch_size = 500
num_batches = len(files) // batch_size

for batch_idx in range(num_batches):
    batch_files = files[batch_idx * batch_size: (batch_idx + 1) * batch_size]
    print("Batch", batch_idx, "Start")
    # Read batch of images
    batch_images, batch_targets = helpers.read_images(batch_files)
    print("Load Done")

    # Predict batch
    batch_mask = model.predict(batch_images)
    print("Mask Done")
    batch_pred = np.argmax(batch_mask, axis=-1)
    print("Pred Done")
    batch_obs = batch_targets[:, :, :, 0]
    print("Obs Done")

    # Calculate batch confusion matrix
    conf_matrix = sklearn.metrics.confusion_matrix(batch_obs.flatten(), batch_pred.flatten(), labels=[0, 1, 2])

    # Add batch confusion matrix to the cumulative confusion matrix
    conf_matrix_cumulative += conf_matrix

    print("Batch", batch_idx, "completed")

# Define the headers and indices
headers = ["Predicted 0", "Predicted 1", "Predicted 2"]
indices = ["Actual 0", "Actual 1", "Actual 2"]

# Print the cumulative confusion matrix with headers and indices
print("\t\t" + "\t".join(headers))
for i in range(len(conf_matrix_cumulative)):
    row_str = "\t".join([str(x) for x in conf_matrix_cumulative[i]])
    print(indices[i] + "\t" + row_str)

# Calculate true positives (TP), false positives (FP), and false negatives (FN) for each class
TP = np.diag(conf_matrix_cumulative)
FP = np.sum(conf_matrix_cumulative, axis=0) - TP
FN = np.sum(conf_matrix_cumulative, axis=1) - TP

# Calculate precision and recall for each class
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print precision and recall for each class
for i in range(len(precision)):
    print("Class", i)
    print("Precision:", precision[i])
    print("Recall:", recall[i])
    print("F1 score:", f1_score[i])
    print()

# # Calculate overall TP, FP, and FN for all classes
# overall_TP = np.sum(TP)
# overall_FP = np.sum(FP)
# overall_FN = np.sum(FN)

# # Calculate overall precision and recall
# overall_precision = overall_TP / (overall_TP + overall_FP)
# overall_recall = overall_TP / (overall_TP + overall_FN)
# overall_f1_score = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall)

# # Print overall precision and recall
# print("Overall Precision:", overall_precision)
# print("Overall Recall:", overall_recall)
# print("Overall F1 score:", overall_f1_score)

# **Additonal Code**

## **Generting Images**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import glob
import helpers
import sklearn.metrics

plt.rcParams['figure.figsize'] = (4, 4)

model = keras.models.load_model('D:/U-Net/Segnet/models/model_segnet_v01')

# Read
files = open('D:/U-Net/Segnet/results/result_21_files_test.txt').read().splitlines()
test_images, test_targets = helpers.read_images(files)

In [ ]:
# For One Exemple
i = 6
image = test_images[i]
plt.imshow(image)
#plt.show()
plt.savefig('D:/U-Net/Segnet/visual_comp/test.png')

target = test_targets[i]
plt.imshow(target)
#plt.show()
plt.savefig('D:/U-Net/Segnet/visual_comp/target.png')

np.expand_dims(image, 0).shape
mask = model.predict(np.expand_dims(image, 0))[0]
pred = np.argmax(mask, axis=-1)
np.unique(pred.flatten())
plt.imshow(pred)
#plt.show()
plt.savefig('D:/U-Net/Segnet/visual_comp/pred.png')

mask = model.predict(test_images)
pred = np.argmax(mask, axis=-1)
np.unique(pred.flatten())

print("Done")

In [ ]:
for i in  range(len(test_images)):
    # RGB
    image = test_images[i]
    plt.imshow(image)
    plt.savefig('D:/U-Net/Segnet/visual_comp/test/test['+ str(i) +'].png')
    # Target
    target = test_targets[i]
    plt.imshow(target)
    plt.savefig('D:/U-Net/Segnet/visual_comp/target/target[' + str(i) + '].png')
    # Pred
    np.expand_dims(image, 0).shape
    mask = model.predict(np.expand_dims(image, 0))[0]
    pred = np.argmax(mask, axis=-1)
    np.unique(pred.flatten())
    plt.imshow(pred)
    plt.savefig('D:/U-Net/Segnet/visual_comp/pred/pred[' + str(i) + '].png')

print("Done")